# DataFrame的连接合并

## **pd.merge**

 pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,left_index=False, 
         right_index=False, sort=True,suffixes=('_x', '_y'), copy=True, indicator=False,validate=None)
**参数如下：**
- left: 拼接的左侧DataFrame对象
- right: 拼接的右侧DataFrame对象
- on: 要加入的列或索引级别名称。 必须在左侧和右侧DataFrame对象中找到。 如果未传递且left_index和right_index为False，则DataFrame中的列的交集将被推断为连接键。
- left_on:左侧DataFrame中的列或索引级别用作键。 可以是列名，索引级名称，也可以是长度等于DataFrame长度的数组。
- right_on: 左侧DataFrame中的列或索引级别用作键。 可以是列名，索引级名称，也可以是长度等于DataFrame长度的数组。
- left_index: 如果为True，则使用左侧DataFrame中的索引（行标签）作为其连接键。 对于具有MultiIndex（分层）的DataFrame，级别数必须与右侧DataFrame中的连接键数相匹配。
- right_index: 与left_index功能相似。
- how: One of ‘left’, ‘right’, ‘outer’, ‘inner’. 默认inner。inner是取交集，outer取并集。比如left：[‘A’,‘B’,‘C’];right[’'A,‘C’,‘D’]；inner取交集的话，left中出现的A会和right中出现的买一个A进行匹配拼接，如果没有是B，在right中没有匹配到，则会丢失。'outer’取并集，出现的A会进行一一匹配，没有同时出现的会将缺失的部分添加缺失值。
- sort: 按字典顺序通过连接键对结果DataFrame进行排序。 默认为True，设置为False将在很多情况下显着提高性能。
- suffixes: 用于重叠列的字符串后缀元组。 默认为（‘x’，’ y’）。
- copy: 始终从传递的DataFrame对象复制数据（默认为True），即使不需要重建索引也是如此。
- indicator:将一列添加到名为_merge的输出DataFrame，其中包含有关每行源的信息。
- \_merge是分类类型，并且对于其合并键仅出现在“左”DataFrame中的观察值，取得值为left_only，对于其合并键仅出现在“右”DataFrame中的观察值为right_only，并且如果在两者中都找到观察点的合并键，则为left_only。

### 基础实例

In [4]:
import pandas as pd

left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                       'A': ['A0', 'A1', 'A2', 'A3'],
                       'B': ['B0', 'B1', 'B2', 'B3']})
right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']})
left

,A,B,key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K2
3,A3,B3,K3


In [5]:
right

,C,D,key
0,C0,D0,K0
1,C1,D1,K1
2,C2,D2,K2
3,C3,D3,K3


In [6]:
#on参数传递的key作为连接键
result = pd.merge(left, right, on='key')
result

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K2,C2,D2
3,A3,B3,K3,C3,D3


### 传入on的参数是列表

In [7]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                      'key2': ['K0', 'K1', 'K0', 'K1'],
                         'A': ['A0', 'A1', 'A2', 'A3'],
                         'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                         'C': ['C0', 'C1', 'C2', 'C3'],
                         'D': ['D0', 'D1', 'D2', 'D3']})

left

,A,B,key1,key2
0,A0,B0,K0,K0
1,A1,B1,K0,K1
2,A2,B2,K1,K0
3,A3,B3,K2,K1


In [8]:
right

,C,D,key1,key2
0,C0,D0,K0,K0
1,C1,D1,K1,K0
2,C2,D2,K1,K0
3,C3,D3,K2,K0


同时传入两个Key，此时会进行以['key1','key2']列表的形式进行对应，left的keys列表是：[['K0', 'K0'],['K0', 'K1'],['K1', 'K0'],['K2', 'K1']],
left的keys列表是：[['K0', 'K0'],['K1', 'K0'],['K1', 'K0'],['K2', 'K0']]，因此会有1个['K0', 'K0']、2个['K1', 'K0']对应。

In [9]:
result = pd.merge(left, right, on=['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A2,B2,K1,K0,C1,D1
2,A2,B2,K1,K0,C2,D2


### Merge method(以左边、右边、取并集，取交集)

In [10]:
result = pd.merge(left, right, how='left', on=['key1', 'key2'])
# Use keys from left frame only 以左边为基准
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A2,B2,K1,K0,C2,D2
4,A3,B3,K2,K1,NaN,NaN


In [11]:
result = pd.merge(left, right, how='right', on=['key1', 'key2'])
# Use keys from right frame only 以右边为基准
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A2,B2,K1,K0,C1,D1
2,A2,B2,K1,K0,C2,D2
3,NaN,NaN,K2,K0,C3,D3


In [12]:
result = pd.merge(left, right, how='outer', on=['key1', 'key2'])
# Use intersection of keys from both frames #取并集
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A2,B2,K1,K0,C2,D2
4,A3,B3,K2,K1,NaN,NaN
5,NaN,NaN,K2,K0,C3,D3


In [14]:
result = pd.merge(left, right, how='inner', on=['key1', 'key2'])
result  #取交集

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A2,B2,K1,K0,C1,D1
2,A2,B2,K1,K0,C2,D2


In [15]:
df1 = pd.DataFrame({'col1': [0, 1], 'col_left':['a', 'b']})
df2 = pd.DataFrame({'col1': [1, 2, 2],'col_right':[2, 2, 2]})
df1

,col1,col_left
0,0,a
1,1,b


In [16]:
df2

,col1,col_right
0,1,2
1,2,2
2,2,2


### 传入indicator参数

**merge接受参数指示符。 如果为True，则将名为_merge的Categorical类型列添加到具有值的输出对象：**

In [17]:
pd.merge(df1, df2, on='col1', how='outer', indicator=True)

,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


In [18]:
pd.merge(df1, df2, on='col1', how='outer', indicator='indicator_column')

,col1,col_left,col_right,indicator_column
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


### 以index为连接键

**需要同时设置left_index= True 和 right_index= True，或者left_index设置的同时，right_on指定某个Key。总的来说就是需要指定left、right链接的键，可以同时是key、index或者混合使用。**

In [26]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                       'B': ['B0', 'B1', 'B2']},
                   index=['K0', 'K1', 'K2'])
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                  'D': ['D0', 'D2', 'D3']},
                    index=['K0', 'K2', 'K3'])
left

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2


In [25]:
right

,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [30]:
# 只有K0、K2有对应的值
pd.merge(left,right,how= 'inner',left_index=True,right_index=True)

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


In [31]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                   'key': ['K0', 'K1', 'K0', 'K1']})
right = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1']},
                    index=['K0', 'K1'])
left

,A,B,key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K0
3,A3,B3,K1


In [32]:
right

,C,D
K0,C0,D0
K1,C1,D1


In [33]:
result = pd.merge(left, right, left_on='key', right_index=True, how='left', sort=False)
#  left_on='key', right_index=True
result

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K0,C0,D0
3,A3,B3,K1,C1,D1


### sort对链接的键值进行排序：

In [34]:
result = pd.merge(left, right, left_on='key', right_index=True, how='left', sort=True)
result

,A,B,key,C,D
0,A0,B0,K0,C0,D0
2,A2,B2,K0,C0,D0
1,A1,B1,K1,C1,D1
3,A3,B3,K1,C1,D1


对于多重索引，目前应用较少，就不做深入学习，以后有需要再加。
总的来说，merge的应用场景是针对链接键来进行操作的，链接键可以是index或者column。但是实际应用时一定注意的是left或者right的键值不要重复，这样引来麻烦

### 左右连接名一样

In [39]:
import pandas as pd
df1=pd.DataFrame({'key':['a','b','a','b','b'],'value1':range(5)})
df2=pd.DataFrame({'key':['a','c','c','c','c'],'value2':range(5)})
display(df1,df2,pd.merge(df1,df2))

,key,value1
0,a,0
1,b,1
2,a,2
3,b,3
4,b,4


,key,value2
0,a,0
1,c,1
2,c,2
3,c,3
4,c,4


,key,value1,value2
0,a,0,0
1,a,2,0


以df1、df2中相同的列名key进行连接,默认how='inner', pd.merge(df1,df2,on='key',how='inner')

### 左右连接键名不一样

In [41]:
df3=pd.DataFrame({'lkey':['a','b','a','b','b'],'data1':range(5)})
df4=pd.DataFrame({'rkey':['a','c','c','c','c'],'data2':range(5)})

In [42]:
df3

,data1,lkey
0,0,a
1,1,b
2,2,a
3,3,b
4,4,b


In [43]:
df4

,data2,rkey
0,0,a
1,1,c
2,2,c
3,3,c
4,4,c


In [44]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')   ### 内连接，默认how='inner'

,data1,lkey,data2,rkey
0,0,a,0,a
1,2,a,0,a


In [46]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey',how='outer')  ### 全连接

,data1,lkey,data2,rkey
0,0.0,a,0.0,a
1,2.0,a,0.0,a
2,1.0,b,NaN,NaN
3,3.0,b,NaN,NaN
4,4.0,b,NaN,NaN
5,NaN,NaN,1.0,c
6,NaN,NaN,2.0,c
7,NaN,NaN,3.0,c
8,NaN,NaN,4.0,c


In [47]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey',how='left')  ### 左连接

,data1,lkey,data2,rkey
0,0,a,0.0,a
1,1,b,NaN,NaN
2,2,a,0.0,a
3,3,b,NaN,NaN
4,4,b,NaN,NaN


In [48]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey',how='right')  ### 左连接

,data1,lkey,data2,rkey
0,0.0,a,0,a
1,2.0,a,0,a
2,NaN,NaN,1,c
3,NaN,NaN,2,c
4,NaN,NaN,3,c
5,NaN,NaN,4,c


### 索引作为连接键

In [49]:
df5=pd.DataFrame(np.arange(12).reshape(3,4),index=list('abc'),columns=['v1','v2','v3','v4'])
df6=pd.DataFrame(np.arange(12,24,1).reshape(3,4),index=list('abd'),columns=['v5','v6','v7','v8'])
df5

,v1,v2,v3,v4
a,0,1,2,3
b,4,5,6,7
c,8,9,10,11


In [50]:
df6

,v5,v6,v7,v8
a,12,13,14,15
b,16,17,18,19
d,20,21,22,23


In [51]:
pd.merge(df5,df6,left_index=True,right_index=True)

,v1,v2,v3,v4,v5,v6,v7,v8
a,0,1,2,3,12,13,14,15
b,4,5,6,7,16,17,18,19


## pd.join

**join 拼接列，主要用于索引上的合并**

join方法提供了一个简便的方法用于将两个DataFrame中的不同的列索引合并成为一个DataFrame

join(self, other, on=None, how='left', lsuffix='', rsuffix='',sort=False):
- 其中参数的意义与merge方法基本相同,只是join方法默认为左外连接how=left
- 默认按索引合并，可以合并相同或相似的索引，不管他们有没有重叠列。
- 可以连接多个DataFrame
- 可以连接除索引外的其他列
- 连接方式用参数how控制
- 通过lsuffix='', rsuffix='' 区分相同列名的列

## pd.concat

**concat可以沿着一条轴将多个对象堆叠到一起**

concat方法相当于数据库中的全连接(UNION ALL),可以指定按某个轴进行连接,也可以指定连接的方式join(outer,inner 只有这两种)。与数据库不同的时concat不会去重，要达到去重的效果可以使用drop_duplicates方法

concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
           keys=None, levels=None, names=None, verify_integrity=False, copy=True):
           
- 轴向连接 pd.concat() 就是单纯地把两个表拼在一起，这个过程也被称作连接（concatenation）、绑定（binding）或堆叠（stacking）。因此可以想见，这个函数的关键参数应该是 axis，用于指定连接的轴向。

- 在默认的 axis=0 情况下，pd.concat([obj1,obj2]) 函数的效果与 obj1.append(obj2) 是相同的；

- 而在 axis=1 的情况下，pd.concat([df1,df2],axis=1) 的效果与 pd.merge(df1,df2,left_index=True,right_index=True,how='outer') 是相同的。
可以理解为 concat 函数使用索引作为“连接键”

pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False)。
- objs 就是需要连接的对象集合，一般是列表或字典；
- axis=0 是连接轴向join='outer' 参数作用于当另一条轴的 index 不重叠的时候，只有 'inner' 和 'outer' 可选（顺带展示 ignore_index=True 的用法）

concat 一些特点：

1.作用于Series时，如果在axis=0时，类似union。axis=1 时，组成一个DataFrame，索引是union后的，列是类似join后的结果。

2.通过参数join_axes=[] 指定自定义索引。

3.通过参数keys=[] 创建层次化索引

4.通过参数ignore_index=True 重建索引。


In [57]:
df1=pd.DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])
df2=pd.DataFrame(np.random.randn(2,3),columns=['b','d','a'])
df1

,a,b,c,d
0,1.781799,-0.904031,-0.195736,0.724643
1,0.063992,-0.404168,-0.098935,1.107169
2,0.115536,-1.052856,0.835261,0.081946


In [58]:
df2

,b,d,a
0,-0.342541,-0.874231,-1.777054
1,1.292261,-0.680519,-0.616739


In [59]:
pd.concat([df1,df2])

,a,b,c,d
0,1.781799,-0.904031,-0.195736,0.724643
1,0.063992,-0.404168,-0.098935,1.107169
2,0.115536,-1.052856,0.835261,0.081946
0,-1.777054,-0.342541,NaN,-0.874231
1,-0.616739,1.292261,NaN,-0.680519


In [56]:
pd.concat([df1,df2],ignore_index=True)

,a,b,c,d
0,-0.896917,-0.521516,-0.842348,0.434458
1,0.932517,-0.934361,-0.231307,-0.356625
2,1.844779,0.940886,0.178891,-0.804834
3,0.232465,1.602990,NaN,0.775265
4,-0.117874,0.204992,NaN,0.601500


In [61]:
df3=pd.DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])
df4=pd.DataFrame(np.random.randn(2,3),columns=['b','d','a'])
pd.concat([df3,df4],axis=1)

,a,b,c,d,b,d,a
0,0.909787,0.618690,-0.307022,-0.763250,-1.138621,-0.094397,-0.155629
1,1.568758,0.230673,0.210328,-0.074905,0.862710,0.231957,-0.690784
2,-0.291872,-0.226892,-0.021042,0.486389,NaN,NaN,NaN


In [62]:
pd.concat([df3,df4],axis=0)

,a,b,c,d
0,0.909787,0.618690,-0.307022,-0.763250
1,1.568758,0.230673,0.210328,-0.074905
2,-0.291872,-0.226892,-0.021042,0.486389
0,-0.155629,-1.138621,NaN,-0.094397
1,-0.690784,0.862710,NaN,0.231957
